# Child Benefit

The Child Benefit is a universal benefit for children administered by HMRC. It is not means-tested, but HMRC does impose a tax charge (the High-Income Tax Charge) on the highest-earning parent if their income is over a threshold to recoup the benefit payment.

Child Benefit parameters can be found in `policyengine_uk/parameters/gov/hmrc/child_benefit` and logic in `policyengine_uk/variables/hmrc/child_benefit.py`.

## Legislation

The Child Benefit is defined primarily in [The Social Security Contributions and Benefits Act 1992, Part IX](https://www.legislation.gov.uk/ukpga/1992/4/part/IX). This defines the core of how the benefit operates, but the rates of payment are prescribed by [The Child Benefit (Rates) Regulations 2006 reg. 2](https://www.legislation.gov.uk/uksi/2006/965/regulation/2), which is amended by up-rating regulations (for example, in 2020, [The Tax Credits, Child Benefit and Guardian’s Allowance Up-rating Regulations 2020](https://www.legislation.gov.uk/uksi/2020/298/regulation/5/made) increased the payment levels).

## Payment levels

Child Benefit defines a (higher) amount for the eldest or only child, and an amount for each additional child. There is no limit on the number of children covered, unlike the Child Tax Credit or the Universal Credit child element. The table below shows the rates covered by PolicyEngine-UK.

In [1]:
from policyengine_uk import CountryTaxBenefitSystem
import pandas as pd

parameters = CountryTaxBenefitSystem().parameters
data = {
    "Date": [],
    "Name": [],
    "Value": [],
    "Reference": [],
}

for parameter in parameters.gov.hmrc.child_benefit.amount.eldest.values_list:
    data["Date"] += [parameter.instant_str]
    data["Name"] += ["Eldest"]
    data["Value"] += [f"£{parameter.value:.2f}"]
    try:
        data["Reference"] += [
            f'<a href="{parameter.metadata["reference"][0]["href"]}">{parameter.metadata["reference"][0]["title"]}</a>'
        ]
    except:
        data["Reference"] += [""]


for (
    parameter
) in parameters.gov.hmrc.child_benefit.amount.additional.values_list:
    data["Date"] += [parameter.instant_str]
    data["Name"] += ["Additional"]
    data["Value"] += [f"£{parameter.value:.2f}"]
    try:
        data["Reference"] += [
            f'<a href="{parameter.metadata["reference"][0]["href"]}">{parameter.metadata["reference"][0]["title"]}</a>'
        ]
    except:
        data["Reference"] += [""]


pd.DataFrame(data).sort_values("Date").set_index(
    ["Date", "Name"]
).style.format(lambda x: x)

## Examples

The below table shows the entitlement calculations for a family of increasing size.

In [2]:
from policyengine_uk import IndividualSim


def get_cb_for_n_children(n):
    sim = IndividualSim(year=2022)
    for i in range(n):
        sim.add_person(age=10, name=str(i))
    sim.add_benunit(members=[str(i) for i in range(n)])
    return f"£{sim.calc('child_benefit').sum():.2f}"


pd.DataFrame(
    {
        "Number of children": list(range(1, 7)),
        "Child Benefit (Annual)": list(
            map(get_cb_for_n_children, range(1, 7))
        ),
    }
).set_index("Number of children")

,Child Benefit (Annual)
Number of children,
1,£1099.80
2,£1827.80
3,£2555.80
4,£3283.80
5,£4011.80
6,£4739.80


## Microsimulation

PolicyEngine-UK simulates Child Benefit as described for each family in the survey microdata, preserving reported takeup. Child Benefit has a relatively high take-up rate compared to other benefits, most recently estimated at 92%. We use the central estimates from 2006 through 2018, provided by HMRC [Child Benefit Statistics: Annual Release, August 2020 Main Commentary | Table 3.1](https://www.gov.uk/government/statistics/child-benefit-statistics-annual-release-august-2020/child-benefit-statistics-annual-release-august-2020-main-commentary), which are calculated from microsimulation and administrative data held by HMRC.